In [18]:
from selenium import webdriver
import urllib.request
import time
import random
import json

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from elasticsearch import Elasticsearch

from tqdm import tqdm

In [19]:
LOCAL = False
es_client = Elasticsearch(hosts=["localhost" if LOCAL else "elasticsearch_tinder"])
#es_client_prod = Elasticsearch(hosts=["localhost" if LOCAL else "elasticsearch"])
es_client.ping()

True

In [4]:
IS_LINUX = False

In [6]:
driver = webdriver.Firefox(executable_path="geckodriver.exe" if not IS_LINUX else "./chromedriver_linux")

##UA = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:25.0) Gecko/20100101 Firefox/25.0'
#PHANTOMJS_ARG = {'phantomjs.page.settings.userAgent': UA}

#driver = webdriver.PhantomJS(desired_capabilities=PHANTOMJS_ARG)

In [7]:
second_driver = webdriver.Firefox()
second_driver.get("https://instagram.com/")
        
insta_button = second_driver.find_element_by_xpath('//*[@class="aOOlW  bIiDR  "]')
insta_button.click()

time.sleep(2)
second_driver.find_element_by_name("username").send_keys("VinLoring@gmail.com")
time.sleep(2)
second_driver.find_element_by_name("password").send_keys("Tinder123")

insta_button_connect = second_driver.find_element_by_xpath('//*[@class="sqdOP  L3NKy   y3zKF     "]').click()

### Connexion 

In [83]:
driver.get("https://tinder.com/app/recs")

In [85]:
classbutton = "button Lts($ls-s) Z(0) CenterAlign Mx(a) Cur(p) Tt(u) Ell Px(24px) Px(20px)--s Py(0) Bdrs(0) Mih(40px) Fw($semibold) focus-button-style Fz($s) Bdrs(4px) Bg(#fff) C($c-pink) Bg($primary-gradient):h C(#fff):h"
python_button = driver.find_element_by_xpath('//*[@class="'+ classbutton +'"]')
python_button.click()

In [86]:
connect_phone_number = "Connexion avec un numéro de tél."
python_button = driver.find_element_by_xpath('//*[@aria-label="'+ connect_phone_number +'"]')
python_button.click()

In [87]:
driver.switch_to.default_content()
driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))

In [88]:
python_button = driver.find_element_by_id("home_children_button")
python_button.click()

### TakeCaptcha
Ignorer cette cell

In [11]:
### IGNORER CETTE CELL
i = 11

img = driver.find_element_by_id("game_challengeItem_image")
src = img.get_attribute('src')
name = "image_0" + str(i) + ".png"
urllib.request.urlretrieve(src, name)
i += 1

filedata = urllib.request.urlopen(src)
datatowrite = filedata.read()
 
with open('/projet/ESGI/3ESGI/NoSQL/BotCapcha/' + name, 'wb') as f:
    f.write(datatowrite)

### IGNORER CETTE CELL

In [90]:
driver.switch_to.default_content()
phone_element = driver.find_element_by_name("phone_number")
phone_element.send_keys("0695776028")

In [91]:
def TakeInstagram():
    try:
        class_desc = "P(16px) Ta(start) Us(t) C($c-secondary) BreakWord Whs(pl) Fz($ms)"
        takedesc = driver.find_element_by_xpath('//*[@class="'+ class_desc +'"]')
        takedesc = takedesc.find_elements_by_css_selector("*")

        insta_list = [ "📷 ", "📸 ", "📷: ", "📸: ", "📷 : ", "📸 : ","@",
            "IG: ", "Ig: ","ig: ", "insta: ", "instagram: ", "Insta: ", "Instagram: ", "INSTA: ", "INSTAGRAM: ",
            "IG, ", "Ig, ","ig, ", "insta, ", "instagram, ", "Insta, ", "Instagram, ", "INSTA, ", "INSTAGRAM: ",
            "IG,", "Ig,","ig,", "insta,", "instagram,", "Insta,", "Instagram,", "INSTA,", "INSTAGRAM,",
            "IG:", "Ig:","ig:", "insta:", "instagram:", "Insta:", "Instagram:", "INSTA:", "INSTAGRAM:",
            "IG : ", "Ig : ", "ig : ", "insta : ", "instagram : ", "Insta : ", "Instagram : ", "INSTA : ", "INSTAGRAM : ",
            "IG :", "Ig :","ig :", "insta :", "instagram :", "Insta :", "Instagram :", "INSTA :", "INSTAGRAM :",
            "IG ", "Ig ","ig ", "insta ", "instagram ", "Insta ", "Instagram ", "INSTA  ", "INSTAGRAM "]

        description = takedesc[0].text
        instagram = ""
        description = description.split("\n")

        for desc_split in description:

            for insta_occurent in insta_list:

                result = desc_split.find(insta_occurent, 0, len(desc_split))
                if result != -1:
                    start = result + len(insta_occurent)
                    instagram = desc_split[start: len(desc_split)]
                    break

        instagramlist = instagram.split(" ")

        for word in instagramlist:
            
            if word != " " and word != "" and word != "➡️":
                instagram = word;
                break
        instagram = instagram.replace("(", "").replace(")", "").replace("/", "")
        return instagram
    
    except:
        
        return ""

In [92]:
def TakeTag():
    try:
        class_tag = "Mt(4px)"
        taketag = driver.find_element_by_xpath('//*[@class="'+ class_tag +'"]')
        taketag = taketag.find_elements_by_css_selector("*")
        
        list_tag = []
        
        for tag in taketag:
            list_tag.append(tag.text)
        
        
        return list_tag
    
    except:
        
        return []

In [93]:
def TakeName():
    try:
        class_name = "Fz($xl) Fw($bold) Fxs(1) Fxw(w) Pend(8px) M(0) D(i)"
        takename = driver.find_element_by_xpath('//*[@class="'+ class_name +'"]')
        
        return takename.text
    
    except:
        
        return ""

In [94]:
def TakeAge():
    try:
        class_age = "Whs(nw) Fz($l)"
        takeage = driver.find_element_by_xpath('//*[@class="'+ class_age +'"]')
        
        return takeage.text
    
    except:
        
        return ""

### INSTAGRAM Picture

In [121]:
def TakePicture(_data):
    try:
    
        second_driver.get("https://instagram.com/" + _data)        
                
        try:
            insta_picture_balise = second_driver.find_element_by_xpath('//*[@class="be6sR"]')
            insta_picture = insta_picture_balise.get_attribute("src")
            return insta_picture
        except:
            pass
        
        insta_picture_balise = second_driver.find_element_by_xpath('//*[@class="_6q-tv"]')
            
        insta_picture = insta_picture_balise.get_attribute("src")
        
        try:
            fail = second_driver.find_element_by_xpath('//*[@class="_7UhW9      x-6xq    qyrsm KV-D4          uL8Hv     l4b0S    "]')
            return ""
        except:
            return insta_picture
    
    except:
        
        return ""

In [1]:
def PushDataBase(data):
    es_client.index(index="tinderplusplus", doc_type='tinder', id=None, body=data)

In [103]:
for i in tqdm(range(250)):
    driver.switch_to.default_content()
    actions = ActionChains(driver)
    actions.send_keys(Keys.ARROW_UP + 't')
    actions.perform()
    insta = TakeInstagram()
    if insta != "":
        tag = TakeTag()
        name = TakeName()
        age = TakeAge()
        picture = TakePicture(insta)

        data = {
            "name": name,
            "instagram": insta,
            "age": age,
            "picture": picture,
            "tag": tag
        }
        #print(data)

        if picture != "":
            PushDataBase(data)

    rnd = random.randrange(1,5)   

    time.sleep(rnd)
    #actions.send_keys(Keys.ARROW_RIGHT + 't')
    actions.send_keys(Keys.ARROW_LEFT + 't')
    actions.perform()   
    time.sleep(2)

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [23:18<00:00,  5.60s/it]


### View Result

In [5]:
es_client.count(index='tinderplusplus', doc_type='tinder')["count"]

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [tinderplusplus]', tinderplusplus, index_or_alias)

In [ ]:
try:
    result = es_client.search(index="tinderplusplus", body={"query": {"match_all": {}}})
    ids = []
    for hit in result['hits']['hits']:
        print("Name : {name}\nAge : {age}\nPicture : {picture} \nInstagram : {instagram} \nTags : {tag} \n".format(**hit['_source']))
        print("******************")
        ids.append(hit["_id"])
except:
     None

### DELETE ALL 

In [15]:
#es_client.indices.delete(index='tinderplusplus')
es_client.count(index='tinderplusplus_prod', doc_type='tinder')["count"]
#es_client.indices.delete(index='tinderplusplus_prod')

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [tinderplusplus_prod]', tinderplusplus_prod, index_or_alias)

### CLEAN DATA

In [55]:
def CleanChar(_data):
    deleteChar= ["(", ")","'", '"', " ",":","[","]"]
    
    for dc in deleteChar:
        _data = _data.replace(dc, "")
        
    return _data

In [13]:
def PushProdDataBase(data):
    es_client.index(index="tinderplusplus_prod", doc_type='tinder', id=None, body=data)

In [7]:
size = es_client.count(index='tinderplusplus', doc_type='tinder')["count"]
try:
    result = es_client.search(index="tinderplusplus", size=size)

    for hit in result['hits']['hits']:
        
        insta = hit['_source']['instagram']
        age = hit['_source']['age']
        name = hit['_source']['name']
        tag = hit['_source']['tag']
        
        insta = CleanChar(insta)
        
        if insta == "":
            print("deleted")
            
        picture = TakePicture(insta)
        
        if picture != "":
            data = {
                "name": name,
                "instagram": insta,
                "age": age,
                "picture": picture,
                "tag": tag
            }
            print(data)
            PushProdDataBase(data)
        else:
            print("deleted")

        time.sleep(3)
        print("******************")
except:
     None

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [tinderplusplus]', tinderplusplus, index_or_alias)

### Extract data

In [4]:
size = es_client.count(index='tinderplusplus_prod', doc_type='tinder')["count"]

result = es_client.search(index="tinderplusplus", size=size)
dataGlobal = []

for hit in result['hits']['hits']:
        
    insta = hit['_source']['instagram']
    age = hit['_source']['age']
    name = hit['_source']['name']
    tag = hit['_source']['tag']     
    picture = hit['_source']['picture']  

    data = {
            "name": name,
            "instagram": insta,
            "age": age,
            "picture": picture,
            "tag": tag
    }
    dataGlobal.append(data)

with open('result.json', 'wt') as fp:
    data_file = json.dumps(dataGlobal)
    fp.write(data_file)

/usr/local/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in count requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [tinderplusplus_prod]', tinderplusplus_prod, index_or_alias)

### Import data

In [20]:
file = open("result.json", "r")

data = file.read()
data_2 = []
data_2 = json.loads(data)

file.close()

for dt in data_2:
    PushProdDataBase(dt)

### CLOSE BROWSER

In [140]:
driver.close()
seconddriver.close()